In [43]:
import pandas as pd
import xgboost as xgb

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score, log_loss, roc_auc_score,classification_report

In [2]:
df = pd.read_csv("../Data/Clean_Data.csv")

In [3]:
feats = ['BATHRM', 'HF_BATHRM', 'HEAT', 'HEAT_D', 'AC', 'NUM_UNITS',
       'ROOMS', 'BEDRM', 'AYB', 'EYB', 'YR_RMDL', 'STORIES', 'SALEDATE',
       'PRICE', 'SALE_NUM', 'GBA', 'STYLE', 'STYLE_D', 'STRUCT',
       'STRUCT_D', 'GRADE', 'GRADE_D', 'CNDTN', 'CNDTN_D', 'EXTWALL',
       'EXTWALL_D', 'ROOF', 'ROOF_D', 'INTWALL', 'INTWALL_D', 'KITCHENS',
       'FIREPLACES', 'USECODE', 'LANDAREA']
#We have removed ROW_ID, BLDG_NUM, GIS_LAST_MOD_DTTM, QUALIFIED, 

x = df[feats]
y = df['QUALIFIED']

In [6]:
#Label Encoding for the String columns
le = preprocessing.LabelEncoder()

for column_name in x.columns:
    if x[column_name].dtype == object:
        x[column_name].fillna('MISSING_DATA', inplace=True)
        x[column_name] = le.fit_transform(x[column_name])
    else:
        x[column_name].fillna(-1, inplace=True)

c:\users\j_m_g\appdata\local\programs\python\python38\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
<ipython-input-6-7e7e17473232>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[column_name] = le.fit_transform(x[column_name])


In [9]:
#Split into test and train datasets
seed = 42

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state = seed)

In [10]:
D_train = xgb.DMatrix(x_train, label=y_train)
D_test = xgb.DMatrix(x_val, label=y_val)

In [11]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 20  # The number of training iterations

In [12]:
model = xgb.train(param, D_train, steps)

In [18]:
preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(y_val, best_preds, average='macro')))
print("Recall = {}".format(recall_score(y_val, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_val, best_preds)))
print(roc_auc_score(y_val, best_preds))

Precision = 0.9050399792253685
Recall = 0.9141921229476635
Accuracy = 0.9030981908698937
0.9141921229476635


In [21]:
clf = xgb.XGBClassifier()
parameters = {
     "eta"    : [0.05, 0.15, 0.25] ,
     "max_depth"        : [ 3, 5, 10, 15],
     "min_child_weight" : [ 1, 4, 8 ],
     "gamma"            : [ 0.0, 0.1, 0.2],
     "colsample_bytree" : [ 0.3 ]
     }

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    scoring="neg_log_loss",
                    cv=3)

grid.fit(x_train, y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None,

In [24]:
print(grid.best_estimator_.get_params)

<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, eta=0.25, gamma=0.0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.25, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)>


In [28]:
y_pred = grid.best_estimator_.predict(x_val)
print(roc_auc_score(y_val, y_pred))

print(classification_report(y_val, y_pred))

0.9223851513872916
              precision    recall  f1-score   support

           0       0.97      0.88      0.92     12940
           1       0.85      0.97      0.91      9557

    accuracy                           0.92     22497
   macro avg       0.91      0.92      0.91     22497
weighted avg       0.92      0.92      0.92     22497



In [40]:
clf = xgb.XGBClassifier()
parameters = {
     "eta"    : [0.2, 0.25, 0.3, 0.35] ,
     "max_depth"        : [ 3, 5, 8, 10],
     "min_child_weight" : [ 1, 2, 3 ],
     "gamma"            : [ 0.0, 0.1, 0.2],
     "colsample_bytree" : [ 0.3, 0.6, 0.8, 1 ],
     "gpu_id": [0]
     }

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    scoring="neg_log_loss",
                    cv=10)

grid.fit(x_train, y_train)

GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_job...
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, va

In [41]:
print(grid.best_estimator_.get_params)

<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, eta=0.25, gamma=0.1,
              gpu_id=0, importance_type='gain', interaction_constraints='',
              learning_rate=0.25, max_delta_step=0, max_depth=5,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)>


In [42]:
y_pred = grid.best_estimator_.predict(x_val)
print(roc_auc_score(y_val, y_pred))

print(classification_report(y_val, y_pred))

0.9237843944225317
              precision    recall  f1-score   support

           0       0.97      0.88      0.92     12940
           1       0.86      0.97      0.91      9557

    accuracy                           0.92     22497
   macro avg       0.92      0.92      0.92     22497
weighted avg       0.92      0.92      0.92     22497



In [59]:
scaler = StandardScaler()

scaler.fit(x)
x_scaler = scaler.transform(x)

In [77]:
#Split into test and train datasets
seed = 42

x_train, x_val, y_train, y_val = train_test_split(x_scaler, y, test_size=0.2, random_state = seed)

In [78]:
model3 = xgb.XGBClassifier(seed=seed, nthread=3)
model3.fit(df_test,y)
print(model3.score(x_train,y_train))

y_pred = model3.predict(x_val)
print(roc_auc_score(y_val, y_pred))

print(classification_report(y_val, y_pred))

0.9377219156845422
0.946697220447637
              precision    recall  f1-score   support

           0       0.98      0.91      0.95      8693
           1       0.89      0.98      0.93      6305

    accuracy                           0.94     14998
   macro avg       0.94      0.95      0.94     14998
weighted avg       0.95      0.94      0.94     14998



In [71]:
pred_df = pd.read_csv("../Data/Clean_Pred_Data.csv")

In [73]:
p = pred_df[feats]

#Label Encoding for the String columns
le = preprocessing.LabelEncoder()

for column_name in x.columns:
    if p[column_name].dtype == object:
        p[column_name].fillna('MISSING_DATA', inplace=True)
        p[column_name] = le.fit_transform(p[column_name])
    else:
        p[column_name].fillna(-1, inplace=True)
        
scaler = StandardScaler()

scaler.fit(p)
p = scaler.transform(p)

c:\users\j_m_g\appdata\local\programs\python\python38\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
<ipython-input-73-b236f6d6f74b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p[column_name] = le.fit_transform(p[column_name])


In [74]:
p

array([[ 1.86442918, -0.98140875, -0.12903025, ...,  4.82634548,
         1.59285236, -0.23718095],
       [ 0.91956129,  0.64442723,  1.0607399 , ...,  2.62134678,
         1.59285236, -0.23718095],
       [-0.97017449, -0.98140875, -0.12903025, ..., -0.68615126,
         1.59285236, -0.29426837],
       ...,
       [ 0.91956129, -0.98140875, -1.31880039, ..., -0.68615126,
         1.44567481,  0.110521  ],
       [-0.0253066 , -0.98140875, -1.31880039, ..., -0.68615126,
         1.44567481,  0.32919718],
       [-0.0253066 , -0.98140875, -1.31880039, ..., -0.68615126,
         1.44567481,  0.22851076]])

In [75]:
p_pred = model3.predict(p)

upload_df = pd.DataFrame()
upload_df['row ID'] = pred_df['row ID']
upload_df['Predict-Qualified'] = p_pred
upload_df.to_csv(r'../Data/XGBoost_Prediction_Clean_Data.csv', index = False)

In [76]:
len(p_pred)

32147